In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 460kB 47.0MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=a926ecbdb8cfb6c5314fb38405e3fe8cf30da520f2a7dbe43339b9d87821ace6
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=084969e40fb54a7a70f3cbd41b0c2dda9bf5f32d2e94cf36c2ab4b9f52fb6ed8
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=8

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import requests
import bs4
from bs4 import BeautifulSoup
import wandb
#from wandb.keras import WandbCallback

In [5]:
wandb.init(project="fake")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


W&B Run: https://app.wandb.ai/abhilashreddys/fake/runs/28kemfbv

In [0]:
df = pd.read_csv('/content/drive/My Drive/fake_news/data.csv')
md = pd.read_csv('/content/drive/My Drive/fake_news/media.csv')

In [0]:
md['class'].describe()

count    1618.000000
mean        1.773177
std         1.312552
min         1.000000
25%         1.000000
50%         1.000000
75%         4.000000
max         4.000000
Name: class, dtype: float64

In [6]:
md.head()

,Unnamed: 0,Source,Url,Bias,Quality,text,class,real
0,0,ABC,https://abcnews.go.com/Politics/us-disrupted-a...,-5.33,52.33,nUS cyber operators disrupted internet access ...,1,1.0
1,1,ABC,https://abcnews.go.com/Politics/appeals-court-...,0.67,51.67,nA federal appeals court rejected the most dir...,1,1.0
2,2,ABC,https://abcnews.go.com/Politics/electoral-coll...,-10.00,32.00,nUS Senator Elizabeth Warren who is competing ...,1,4.0
3,3,ABC,https://abcnews.go.com/Politics/facebook-agree...,-2.33,52.33,nFacebook announced Tuesday that it will block...,1,1.0
4,4,ABC,https://abcnews.go.com/Politics/donald-trump-t...,-4.33,52.67,nPresident Donald Trump and the Trump of the T...,1,1.0


In [0]:

# md['real'] = None
# for i in range(md.shape[0]):
#     if(md.at[i,'Quality']<32):
#         md.at[i,'real'] = 4
#     elif (md.at[i,'Quality']==32 and abs(md.at[i,'Bias'])<5.0):
#         md.at[i,'real'] = 1
#     elif (md.at[i,'Quality']==32 and abs(md.at[i,'Bias'])>5.0):
#         md.at[i,'real'] = 4
#     elif (md.at[i,'Quality']>32 and abs(md.at[i,'Bias'])<15.0):
#         md.at[i,'real'] = 1
#     elif (md.at[i,'Quality']>32 and abs(md.at[i,'Bias'])>15.0):
#         md.at[i,'real'] = 4

In [0]:
# md.to_csv('/content/drive/My Drive/fake_news/media.csv',index=False)

In [0]:
md['real'].value_counts()

1    1068
4     547
Name: real, dtype: int64

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['class'], test_size=0.15, random_state=42)
X1_train, X1_test, y1_train, y1_test = train_test_split(md['text'], md['real'], test_size=0.2, random_state=42)

In [8]:
X_train.head()

86     We great country  We better  We must embrace F...
353    Washington DC Officen713 Hart Senate Office Bu...
429    nI cannot guarantee going first 100 days  But ...
174    May 23 2010 x97  nTAPPER Hello again  Joining ...
626    More 2000 highranking officials Chinese Commun...
Name: text, dtype: object

In [0]:
train_data = [{'text': text, 'type': type_data } for text in X_train.to_list()+X1_train.to_list() for type_data in y_train.to_list()+y1_train.to_list()]
test_data = [{'text': text, 'type': type_data } for text in X_test.to_list()+X1_test.to_list() for type_data in y_test.to_list()+y1_test.to_list()]

In [0]:
train_texts = X_train.to_list()+X1_train.to_list()
train_labels = y_train.to_list()+y1_train.to_list()
test_texts = X_test.to_list()+X1_test.to_list()
test_labels = y_test.to_list()+y1_test.to_list()

In [0]:
len(train_texts)

1866

In [11]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 2.8MB/s 


In [12]:
!pip install transformers

     |████████████████████████████████| 501kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 49.1MB/s 
     |████████████████████████████████| 870kB 41.3MB/s 
     |████████████████████████████████| 3.7MB 28.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=0cdbdfd577223257bbeea1d3536e74877f1e4aeaa090804b986771bbceb11ae4
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [13]:
import torch.nn as nn
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
#from pytorch_pretrained_bert import AlbertTokenizer, AlbertModel
from transformers import AlbertTokenizer, AlbertModel
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:255], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:255], test_texts))
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=256, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=256, truncating="post", padding="post", dtype="int")

100%|██████████| 231508/231508 [00:00<00:00, 422990.52B/s]


In [0]:
print(train_tokens_ids[0])

[  101  2057  2307  2406  2057  2488  2057  2442  9979  8759 11174 19852
  7072  3961  2995  4889  6481  5367  8574  2086  3283  4225  2231  1523
  1997  2111  2111  2111  1524  2009  4212  2149  2507  2440  3574  2231
  1523  2011  2111  1524  2691  9866  7686  2120  6043 18684  2692  2651
  1521  1055  9849  2592  4807  2974  3073  2149 15741  3754 10639  4480
 11158  2576  2832  7337 14245  4480  2377  3622  2535  3136  2231  2078
  2053  3126  2406  3791 14524  1516  1516 14524  3327  6043  3327  2111
  7072  2993  1045  2903 19519  2110  2576  7344  3370  8388 14524 25405
  1523  2120  6349  7072  1524  1996  2120  6349  3818  6094  8628  1045
  2764  2197  5417  2086 18684  2692  1996  2120  6349  1516  1516  2043
 11955  2375  1516  1516  2097  3288  2137  4480  3136  2231  2375 12088
  2053  3126  2093  5628  2231  2468  2066 14480  3242  2093  3456  1996
  8759  2071 18035  2172  2769  2569  5426  2052 13593  2576  2832  3228
  2149  1516  1516  2137  4480  1516  1516  4884  2

In [0]:
# for masking
train_y = np.array(train_labels) == 4
test_y = np.array(test_labels) == 4

In [0]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)

In [17]:
torch.cuda.is_available()

True

In [0]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [0]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)#.to(device)
test_masks_tensor = torch.tensor(test_masks)#.to(device)

In [0]:
len(train_masks)

1866

In [0]:
train_tokens_tensor = torch.tensor(train_tokens_ids)#.to(device)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
train_y_tensor = train_y_tensor#.to(device)
test_tokens_tensor = torch.tensor(test_tokens_ids)#.to(device)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()
test_y_tensor = test_y_tensor#.to(device)

In [21]:
print(test_tokens_tensor.shape)

torch.Size([425, 256])


In [0]:
BATCH_SIZE = 4
EPOCHS = 5

In [0]:
config = wandb.config

In [0]:
config.epochs = 5
config.batches_per_epoch = 5

In [0]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [0]:
type(test_dataloader)

torch.utils.data.dataloader.DataLoader

In [26]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()
wandb.watch(bert_clf)
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        token_ids, masks, labels = token_ids.to(device), masks.to(device), labels.to(device)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        wandb.log({"Training loss": train_loss})
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

100%|██████████| 407873900/407873900 [00:37<00:00, 11000984.95B/s]


Epoch:  1
0/870489.0 loss: 0.7249943017959595 
Epoch:  1
1/870489.0 loss: 0.7037903368473053 
Epoch:  1
2/870489.0 loss: 0.6715389291445414 
Epoch:  1
3/870489.0 loss: 0.6621959358453751 
Epoch:  1
4/870489.0 loss: 0.6514493823051453 
Epoch:  1
5/870489.0 loss: 0.6822243730227152 
Epoch:  1
6/870489.0 loss: 0.6886430297579084 
Epoch:  1
7/870489.0 loss: 0.6799551546573639 
Epoch:  1
8/870489.0 loss: 0.6952763795852661 
Epoch:  1
9/870489.0 loss: 0.6802936315536499 
Epoch:  1
10/870489.0 loss: 0.6909605156291615 
Epoch:  1
11/870489.0 loss: 0.6794828027486801 
Epoch:  1
12/870489.0 loss: 0.6831046801346999 
Epoch:  1
13/870489.0 loss: 0.6730569005012512 
Epoch:  1
14/870489.0 loss: 0.6787388285001119 
Epoch:  1
15/870489.0 loss: 0.6823414377868176 
Epoch:  1
16/870489.0 loss: 0.6820440187173731 
Epoch:  1
17/870489.0 loss: 0.6844271620114645 
Epoch:  1
18/870489.0 loss: 0.681160757416173 
Epoch:  1
19/870489.0 loss: 0.6793746054172516 
Epoch:  1
20/870489.0 loss: 0.6734367694173541 
Epo

In [28]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    test_loss = 0
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        token_ids, masks, labels = token_ids.to(device), masks.to(device), labels.to(device)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        test_loss += loss.item()
        numpy_logits = logits.cpu().detach().numpy()
        #print(numpy_logits)
        wandb.log({"Testing loss": test_loss})
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.67      0.95      0.79       230
        True       0.88      0.46      0.61       195

    accuracy                           0.72       425
   macro avg       0.78      0.70      0.70       425
weighted avg       0.77      0.72      0.70       425



In [0]:
# torch.save({
#             'epoch': EPOCHS,
#             'model_state_dict': bert_clf.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': loss,
#             }, '/content/weights1.pth')

In [0]:
torch.save(bert_clf.state_dict(), '/content/drive/My Drive/fake_news/nb_state256.pth') #only 256 has trained on media data

NameError: ignored

In [0]:
#!mv '/content/weights1.pth' '/content/drive/My Drive/fake_news'

In [0]:
#!mv '/content/data.csv' '/content/drive/My Drive/fake_news'

In [0]:
# !mv '/content/state256.pth' '/content/drive/My Drive/fake_news'

In [0]:
# test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:255], [article]))
# test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
# test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=256, truncating="post", padding="post", dtype="int")
# test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
# test_masks_tensor = torch.tensor(test_masks)
# test_tokens_ids = torch.tensor(test_tokens_ids)

In [0]:
article = md.at[435,'text']
article = str(article)
# path = '/content/drive/My Drive/fake_news/state1.pth'

In [0]:
md.at[435,'class']

4

In [0]:
print([article])

['Sign in to your Forbes accountxa0or register For instructions on how to disable your ad blockerxa0click here If this is your first time registering please check your inbox for more information about the benefits of your Forbes account and what you can do next Andrew Siegel EVP of Strategy and Corporate Development for Advance Publications is seen at the + SELF X Clinton Foundation What Works for Women Leaders in Technology panel on Thursday Oct 29 2015 in New York New York Brian AchAP Images for SELF As of January 2019 only 24 companies in the SP 500 had CEOs that were women  thats 48 Catalyst 2019 Inxa0Emily Bazelons New York Times article A Head at the Seat of the Table Kathryn Phillips PhD described the doublebind women face in the workplace If they’re perceived as nice and warm and nurturing as they’re expected to be they don’t show what it takes to move into a leadership position But when they take charge to get things done they’re often seen as angrier or more aggressive than m

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 23328246.45B/s]


In [0]:
def Punctuation(string): 
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    # return string without punctuation 
    return string

In [0]:
def get_text(url):
    try:
        result=requests.get(str(url))
    except Exception:
        print("error in scraping url")
        return None
    src=result.content
    soup=BeautifulSoup(src,'lxml')   
    text=[]	
    for p_tag in soup.find_all('p'):
        text.append(p_tag.text)
    text = Punctuation(str(text))
    return text

In [0]:
article = get_text('https://babylonbee.com/news/podiums-at-next-debate-to-be-equipped-with-life-alert-buttons')

In [0]:
path='/content/drive/My Drive/fake_news/nb_state256.pth'
model = BertBinaryClassifier()
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)
model.load_state_dict(torch.load(path))
model.eval()

100%|██████████| 407873900/407873900 [00:05<00:00, 76573217.13B/s]


BertBinaryClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            

In [0]:
def test(article,model):
    # model = model
    # optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)
    # model.load_state_dict(torch.load(path))
    # checkpoint = torch.load(path)
    # model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # epoch = checkpoint['epoch']
    # loss = checkpoint['loss']
    # model.eval()
    bert_predicted = []
    all_logits = []
    test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:255], [article]))
    test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
    test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=256, truncating="post", padding="post", dtype="int")
    test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
    test_masks_tensor = torch.tensor(test_masks)
    test_tokens_ids = torch.tensor(test_tokens_ids)
    # test_masks_tensor = test_masks_tensor.to(device)
    # test_tokens_ids = test_tokens_ids.to(device)
    with torch.no_grad():
        logits = model(test_tokens_ids, test_masks_tensor)
        #loss_func = nn.BCELoss()
        #loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        if(numpy_logits[0,0] > 0.5):
            return 4
        else:
            return 1

In [0]:
print(test(article,model))

4


In [0]:
def answer(url,model):
    article = get_text(url)
    ans = test(article,model)
    return ans

In [0]:
url = 'https://babylonbee.com/news/podiums-at-next-debate-to-be-equipped-with-life-alert-buttons'
print(answer(url,model))

4
